# Import Libraries

In [1]:
import pandas as pd
import numpy as np

from bs4 import BeautifulSoup
import requests

import re

import unicodedata

import warnings
warnings.filterwarnings('ignore')

import time

# Datapath

In [2]:
dPath = "D:/Data/cycling/"

# Define Functions

## Rider information

In [3]:
# Function to WebScrape table from url
def getTable(url):
    source_sf = requests.get(url).text
    soup = BeautifulSoup(source_sf, 'lxml')
    table = soup.find('table')
    html_tables = pd.read_html(table.prettify())
    
    return html_tables

In [4]:
# Function to WebScrape the rider height from url
def getHeight(url):
    source_sf = requests.get(url).text
    soup = BeautifulSoup(source_sf, 'lxml')
    try:
        height = soup.find('b',text='Height:').next_sibling
        height = height.split("m")[0]
        height = height.replace(" ","")
    except IndexError:
        height = "Unknown"
    except AttributeError:
        height = "Unknown"
    
    return height

# IndexError implies website was found but no height value could be obtained
# AttributeError implies url could not be reached

In [5]:
# Function to WebScrape the rider weight from url
def getWeight(url):
    source_sf = requests.get(url).text
    soup = BeautifulSoup(source_sf, 'lxml')
    try:
        weight = soup.find('b',text='Weight:').next_sibling
        weight = weight.split("kg")[0]
        weight = weight.replace(" ","")
    except IndexError:
        weight = "Unknown"
    except AttributeError:
        weight = "Unknown"
    
    return weight

# IndexError implies website was found but no height value could be obtained
# AttributeError implies url could not be reached

In [6]:
# Function to WebScrape the rider height from url
def getNat(url):
    source_sf = requests.get(url).text
    soup = BeautifulSoup(source_sf, 'lxml')
    try:
        nat = soup.find('b',text='Nationality:').find_next_sibling('a').text
    except IndexError:
        nat = "Unknown"
    except AttributeError:
        nat = "Unknown"
    
    return nat

# IndexError implies website was found but no nationality value could be obtained
# AttributeError implies url could not be reached

In [7]:
def classify_races(rider_wins):
    rider_wins["Class"] = rider_wins["Class"].astype(str)
    # Replace some class names
    rider_wins["Class"] = rider_wins["Class"].replace("1.7.1","1.2")
    rider_wins["Class"] = rider_wins["Class"].replace("HIS","2.UWT")

    rider_wins["Class"] = rider_wins["Class"].str.replace("Pro","HC")
    rider_wins["Class"] = rider_wins["Class"].str.replace("PT","HC")
    rider_wins["Class"] = rider_wins["Class"].str.replace("HC2","HC")

    rider_wins["Class"] = rider_wins["Class"].str.replace("1.Pro","1.HC")
    rider_wins["Class"] = rider_wins["Class"].str.replace("2.Pro","2.HC")
    rider_wins["Class"] = rider_wins["Class"].str.replace("HC2","2.HC")

    rider_wins["Class"]=np.where(rider_wins["Race"].str.contains("Vuelta a España"), "2.UWT", rider_wins["Class"])
    rider_wins["Class"]=np.where(rider_wins["Race"].str.contains("Giro d'Italia"), "2.UWT", rider_wins["Class"])
    rider_wins["Class"]=np.where(rider_wins["Race"].str.contains("Tour de France"), "2.UWT", rider_wins["Class"])

    rider_wins["Class"]=np.where(rider_wins["Race"].str.contains("Vuelta Ciclista al Pais Vasco"), "2.UWT", rider_wins["Class"])

    
    rider_wins["Class"]=np.where((rider_wins["Class"]=="UWT") & (rider_wins["Race"].str.contains("Stage")), "2.UWT", rider_wins["Class"])
    rider_wins["Class"]=np.where((rider_wins["Class"]=="UWT") & (rider_wins["Race"].str.contains("Prologue")), "2.UWT", rider_wins["Class"])

  
    # Where the class is outdated, replace the class with correct values (looked up manually by myself)

    rider_wins["Class"]=np.where((rider_wins["Class"]=="HC") & (rider_wins["Race"].str.contains("Stage")), "2.HC", rider_wins["Class"])
    rider_wins["Class"]=np.where((rider_wins["Class"]=="HC") & (rider_wins["Race"].str.contains("Prologue")), "2.HC", rider_wins["Class"])

    #1.HC
    rider_wins["Class"]=np.where((rider_wins["Class"]=="HC") & (rider_wins["Race"]=="Paris - Tours"), "1.HC", rider_wins["Class"])
    rider_wins["Class"]=np.where((rider_wins["Class"]=="HC") & (rider_wins["Race"]=="Amstel Gold Race"), "1.HC", rider_wins["Class"])
    rider_wins["Class"]=np.where((rider_wins["Class"]=="HC") & (rider_wins["Race"]=="Milano-Sanremo"), "1.HC", rider_wins["Class"])
    rider_wins["Class"]=np.where((rider_wins["Class"]=="HC") & (rider_wins["Race"]=="Gent - Wevelgem"), "1.HC", rider_wins["Class"])
    rider_wins["Class"]=np.where((rider_wins["Class"]=="HC") & (rider_wins["Race"]=="Ronde van Vlaanderen / Tour des Flandres"), "1.HC", rider_wins["Class"])
    rider_wins["Class"]=np.where((rider_wins["Class"]=="HC") & (rider_wins["Race"]=="Paris - Roubaix"), "1.HC", rider_wins["Class"])
    rider_wins["Class"]=np.where((rider_wins["Class"]=="HC") & (rider_wins["Race"]=="Grand Prix Cycliste de Quebec"), "1.HC", rider_wins["Class"])
    rider_wins["Class"]=np.where((rider_wins["Class"]=="HC") & (rider_wins["Race"]=="GP Ouest France - Plouay"), "1.HC", rider_wins["Class"])
    rider_wins["Class"]=np.where((rider_wins["Class"]=="HC") & (rider_wins["Race"]=="Liège - Bastogne - Liège"), "1.HC", rider_wins["Class"])
    rider_wins["Class"]=np.where((rider_wins["Class"]=="HC") & (rider_wins["Race"]=="La Flèche Wallonne"), "1.HC", rider_wins["Class"])
    rider_wins["Class"]=np.where((rider_wins["Class"]=="HC") & (rider_wins["Race"]=="Vattenfall Cyclassics"), "1.HC", rider_wins["Class"])
    rider_wins["Class"]=np.where((rider_wins["Class"]=="HC") & (rider_wins["Race"]=="Grand Prix Cycliste de Montréal"), "1.HC", rider_wins["Class"])

    rider_wins["Class"]=np.where((rider_wins["Class"].isna()) & (rider_wins["Race"]=="Paris - Tours"), "1.HC", rider_wins["Class"])
    rider_wins["Class"]=np.where((rider_wins["Class"].isna()) & (rider_wins["Race"]=="Amstel Gold Race"), "1.HC", rider_wins["Class"])
    rider_wins["Class"]=np.where((rider_wins["Class"].isna()) & (rider_wins["Race"]=="Milano-Sanremo"), "1.HC", rider_wins["Class"])
    rider_wins["Class"]=np.where((rider_wins["Class"].isna()) & (rider_wins["Race"]=="Gent - Wevelgem"), "1.HC", rider_wins["Class"])
    rider_wins["Class"]=np.where((rider_wins["Class"].isna()) & (rider_wins["Race"]=="Ronde van Vlaanderen / Tour des Flandres"), "1.HC", rider_wins["Class"])
    rider_wins["Class"]=np.where((rider_wins["Class"].isna()) & (rider_wins["Race"]=="Paris - Roubaix"), "1.HC", rider_wins["Class"])
    rider_wins["Class"]=np.where((rider_wins["Class"].isna()) & (rider_wins["Race"]=="Grand Prix Cycliste de Quebec"), "1.HC", rider_wins["Class"])
    rider_wins["Class"]=np.where((rider_wins["Class"].isna()) & (rider_wins["Race"]=="GP Ouest France - Plouay"), "1.HC", rider_wins["Class"])
    rider_wins["Class"]=np.where((rider_wins["Class"].isna()) & (rider_wins["Race"]=="Liège - Bastogne - Liège"), "1.HC", rider_wins["Class"])
    rider_wins["Class"]=np.where((rider_wins["Class"].isna()) & (rider_wins["Race"]=="La Flèche Wallonne"), "1.HC", rider_wins["Class"])
    rider_wins["Class"]=np.where((rider_wins["Class"].isna()) & (rider_wins["Race"]=="Vattenfall Cyclassics"), "1.HC", rider_wins["Class"])
    rider_wins["Class"]=np.where((rider_wins["Class"].isna()) & (rider_wins["Race"]=="Grand Prix Cycliste de Montréal"), "1.HC", rider_wins["Class"])

    #2.HC
    rider_wins["Class"]=np.where((rider_wins["Class"]=="HC") & (rider_wins["Race"]=="Critérium du Dauphiné Libéré"), "2.HC", rider_wins["Class"])
    rider_wins["Class"]=np.where((rider_wins["Class"]=="HC") & (rider_wins["Race"]=="Deutschland Tour"), "2.HC", rider_wins["Class"])
    rider_wins["Class"]=np.where((rider_wins["Class"]=="HC") & (rider_wins["Race"]=="Tour de Pologne"), "2.HC", rider_wins["Class"])
    rider_wins["Class"]=np.where((rider_wins["Class"]=="HC") & (rider_wins["Race"]=="Clasica Ciclista San Sebastian"), "2.HC", rider_wins["Class"])
    rider_wins["Class"]=np.where((rider_wins["Class"]=="HC") & (rider_wins["Race"].str.contains("Tour de Suisse")), "2.HC", rider_wins["Class"])
    rider_wins["Class"]=np.where((rider_wins["Class"]=="HC") & (rider_wins["Race"]=="Tour of Germany"), "2.HC", rider_wins["Class"])
    rider_wins["Class"]=np.where((rider_wins["Class"]=="HC") & (rider_wins["Race"]=="Volta Ciclista a Catalunya"), "2.HC", rider_wins["Class"])
    rider_wins["Class"]=np.where((rider_wins["Class"]=="HC") & (rider_wins["Race"]=="Tour de Romandie"), "2.HC", rider_wins["Class"])
    rider_wins["Class"]=np.where((rider_wins["Class"]=="HC") & (rider_wins["Race"]=="Critérium du Dauphiné"), "2.HC", rider_wins["Class"])
    rider_wins["Class"]=np.where((rider_wins["Class"]=="HC") & (rider_wins["Race"]=="Tour Down Under"), "2.HC", rider_wins["Class"])

    rider_wins["Class"]=np.where((rider_wins["Class"].isna()) & (rider_wins["Race"]=="Critérium du Dauphiné Libéré"), "2.HC", rider_wins["Class"])
    rider_wins["Class"]=np.where((rider_wins["Class"].isna()) & (rider_wins["Race"]=="Deutschland Tour"), "2.HC", rider_wins["Class"])
    rider_wins["Class"]=np.where((rider_wins["Class"].isna()) & (rider_wins["Race"]=="Tour de Pologne"), "2.HC", rider_wins["Class"])
    rider_wins["Class"]=np.where((rider_wins["Class"].isna()) & (rider_wins["Race"]=="Clasica Ciclista San Sebastian"), "2.HC", rider_wins["Class"])
    rider_wins["Class"]=np.where((rider_wins["Class"].isna()) & (rider_wins["Race"]=="Tour de Suisse"), "2.HC", rider_wins["Class"])
    rider_wins["Class"]=np.where((rider_wins["Class"].isna()) & (rider_wins["Race"]=="Tour of Germany"), "2.HC", rider_wins["Class"])
    rider_wins["Class"]=np.where((rider_wins["Class"].isna()) & (rider_wins["Race"]=="Volta Ciclista a Catalunya"), "2.HC", rider_wins["Class"])
    rider_wins["Class"]=np.where((rider_wins["Class"].isna()) & (rider_wins["Race"]=="Tour de Romandie"), "2.HC", rider_wins["Class"])
    rider_wins["Class"]=np.where((rider_wins["Class"].isna()) & (rider_wins["Race"]=="Critérium du Dauphiné"), "2.HC", rider_wins["Class"])
    rider_wins["Class"]=np.where((rider_wins["Class"].isna()) & (rider_wins["Race"]=="Tour Down Under"), "2.HC", rider_wins["Class"])

    rider_wins["Class"]=np.where((rider_wins["Class"]=="WT") & (rider_wins["Race"].str.contains("Stage")), "2.UWT", rider_wins["Class"])
    rider_wins["Class"]=np.where((rider_wins["Class"]=="WT") & (rider_wins["Race"].str.contains("Prologue")), "2.UWT", rider_wins["Class"])

    # 1.UWT
    rider_wins["Class"]=np.where((rider_wins["Class"]=="WT") & (rider_wins["Race"]=="GP Ouest France - Plouay"), "1.UWT", rider_wins["Class"])
    rider_wins["Class"]=np.where((rider_wins["Class"]=="WT") & (rider_wins["Race"]=="HEW Cyclassics"), "1.UWT", rider_wins["Class"])
    rider_wins["Class"]=np.where((rider_wins["Class"]=="WT") & (rider_wins["Race"]=="Vattenfall Cyclassics"), "1.UWT", rider_wins["Class"])
    rider_wins["Class"]=np.where((rider_wins["Class"]=="WT") & (rider_wins["Race"]=="Liège - Bastogne - Liège"), "1.UWT", rider_wins["Class"])
    rider_wins["Class"]=np.where((rider_wins["Class"]=="WT") & (rider_wins["Race"]=="Gent - Wevelgem"), "1.UWT", rider_wins["Class"])
    rider_wins["Class"]=np.where((rider_wins["Class"]=="WT") & (rider_wins["Race"]=="E3 Prijs Vlaanderen"), "1.UWT", rider_wins["Class"])
    rider_wins["Class"]=np.where((rider_wins["Class"]=="WT") & (rider_wins["Race"]=="Milano-Sanremo"), "1.UWT", rider_wins["Class"])
    rider_wins["Class"]=np.where((rider_wins["Class"]=="WT") & (rider_wins["Race"]=="E3 Prijs Vlaanderen - Harelbeke"), "1.UWT", rider_wins["Class"])
    rider_wins["Class"]=np.where((rider_wins["Class"]=="WT") & (rider_wins["Race"]=="La Flèche Wallonne"), "1.UWT", rider_wins["Class"])
    rider_wins["Class"]=np.where((rider_wins["Class"]=="WT") & (rider_wins["Race"]=="Amstel Gold Race"), "1.UWT", rider_wins["Class"])
    rider_wins["Class"]=np.where((rider_wins["Class"]=="WT") & (rider_wins["Race"]=="Giro di Lombardia"), "1.UWT", rider_wins["Class"])
    rider_wins["Class"]=np.where((rider_wins["Class"]=="WT") & (rider_wins["Race"]=="Grand Prix Cycliste de Montréal"), "1.UWT", rider_wins["Class"])
    rider_wins["Class"]=np.where((rider_wins["Class"]=="WT") & (rider_wins["Race"]=="Grand Prix Cycliste de Québec"), "1.UWT", rider_wins["Class"])
    rider_wins["Class"]=np.where((rider_wins["Class"]=="WT") & (rider_wins["Race"]=="Il Lombardia"), "1.UWT", rider_wins["Class"])
    rider_wins["Class"]=np.where((rider_wins["Class"]=="WT") & (rider_wins["Race"]=="Ronde van Vlaanderen / Tour des Flandres"), "1.UWT", rider_wins["Class"])
    rider_wins["Class"]=np.where((rider_wins["Class"]=="WT") & (rider_wins["Race"]=="Paris - Roubaix"), "1.UWT", rider_wins["Class"])

    # 2.UWT
    rider_wins["Class"]=np.where((rider_wins["Class"]=="WT") & (rider_wins["Race"]=="Tirreno-Adriatico"), "2.UWT", rider_wins["Class"])
    rider_wins["Class"]=np.where((rider_wins["Class"]=="WT") & (rider_wins["Race"]=="Tour de Pologne"), "2.UWT", rider_wins["Class"])
    rider_wins["Class"]=np.where((rider_wins["Class"]=="WT") & (rider_wins["Race"]=="Tour de Suisse"), "2.UWT", rider_wins["Class"])
    rider_wins["Class"]=np.where((rider_wins["Class"]=="WT") & (rider_wins["Race"]=="Santos Tour Down Under"), "2.UWT", rider_wins["Class"])
    rider_wins["Class"]=np.where((rider_wins["Class"]=="WT") & (rider_wins["Race"]=="Tour Down Under"), "2.UWT", rider_wins["Class"])
    rider_wins["Class"]=np.where((rider_wins["Class"]=="WT") & (rider_wins["Race"]=="Tour of Beijing"), "2.UWT", rider_wins["Class"])
    rider_wins["Class"]=np.where((rider_wins["Class"]=="WT") & (rider_wins["Race"]=="Critérium du Dauphiné"), "2.UWT", rider_wins["Class"])
    rider_wins["Class"]=np.where((rider_wins["Class"]=="WT") & (rider_wins["Race"]=="Tour de Romandie"), "2.UWT", rider_wins["Class"])
    rider_wins["Class"]=np.where((rider_wins["Class"]=="WT") & (rider_wins["Race"]=="Volta Ciclista a Catalunya"), "2.UWT", rider_wins["Class"])
    rider_wins["Class"]=np.where((rider_wins["Class"]=="WT") & (rider_wins["Race"]=="Clasica Ciclista San Sebastian"), "2.UWT", rider_wins["Class"])

    # Where the class is unknown, replace the class with correct values (looked up manually by myself)

    rider_wins["Class"]=np.where((rider_wins["Class"].isna()) & (rider_wins["Race"].str.contains("Classique des Alpes")), "1.UWT", rider_wins["Class"])
    rider_wins["Class"]=np.where((rider_wins["Class"].isna()) & (rider_wins["Race"].str.contains("Rund um die Hainleite-Erfurt")), "1.1", rider_wins["Class"])
    rider_wins["Class"]=np.where((rider_wins["Class"].isna()) & (rider_wins["Race"].str.contains("Continentale Classic")), "1.1", rider_wins["Class"])
    rider_wins["Class"]=np.where((rider_wins["Class"].isna()) & (rider_wins["Race"].str.contains("Bordeaux - Caudéran")), "1.1", rider_wins["Class"])
    rider_wins["Class"]=np.where((rider_wins["Class"].isna()) & (rider_wins["Race"].str.contains("Tour du Languedoc-Rousillon")), "2.1", rider_wins["Class"])
    rider_wins["Class"]=np.where((rider_wins["Class"].isna()) & (rider_wins["Race"].str.contains("Tour de Lorraine")), "2.2", rider_wins["Class"])
    rider_wins["Class"]=np.where((rider_wins["Class"].isna()) & (rider_wins["Race"].str.contains("La Tropicale Amissa Bongo Ondimb")), "2.1", rider_wins["Class"])
    rider_wins["Class"]=np.where((rider_wins["Class"].isna()) & (rider_wins["Race"].str.contains("Tour of Sweden")), "2.1", rider_wins["Class"])
    rider_wins["Class"]=np.where((rider_wins["Class"].isna()) & (rider_wins["Race"].str.contains("Brussel-Ingooigem")), "1.1", rider_wins["Class"])
    rider_wins["Class"]=np.where((rider_wins["Class"].isna()) & (rider_wins["Race"].str.contains("GP Eddy Merckx")), "1.1", rider_wins["Class"])
    rider_wins["Class"]=np.where((rider_wins["Class"].isna()) & (rider_wins["Race"].str.contains("US Pro Championship")), "1.1", rider_wins["Class"])
    rider_wins["Class"]=np.where((rider_wins["Class"].isna()) & (rider_wins["Race"].str.contains("Trofeo Città di Borgomanero")), "1.1", rider_wins["Class"])
    rider_wins["Class"]=np.where((rider_wins["Class"].isna()) & (rider_wins["Race"].str.contains("GP Citta di Rio Saliceto e Correggi")), "1.1", rider_wins["Class"])
    rider_wins["Class"]=np.where((rider_wins["Class"].isna()) & (rider_wins["Race"].str.contains("La Roue Tourangelle Région Centre - Classic Loire Touraine Vignobles & Chateaux")), "1.1", rider_wins["Class"])
    rider_wins["Class"]=np.where((rider_wins["Class"].isna()) & (rider_wins["Race"].str.contains("International Bayern Rundfahrt")), "2.HC", rider_wins["Class"])
    rider_wins["Class"]=np.where((rider_wins["Class"].isna()) & (rider_wins["Race"].str.contains("International Bayern Rundfarht")), "2.HC", rider_wins["Class"])
    rider_wins["Class"]=np.where((rider_wins["Class"].isna()) & (rider_wins["Race"].str.contains("Manx Premier Trophy")), "1.2", rider_wins["Class"])
    rider_wins["Class"]=np.where((rider_wins["Class"].isna()) & (rider_wins["Race"].str.contains("Wiesbauer Tour/Tour of Austria")), "2.1", rider_wins["Class"])
    rider_wins["Class"]=np.where((rider_wins["Class"].isna()) & (rider_wins["Race"].str.contains("Österreich-Rundfahrt")), "2.1", rider_wins["Class"])
    rider_wins["Class"]=np.where((rider_wins["Class"].isna()) & (rider_wins["Race"].str.contains("Rundfahrt")), "2.1", rider_wins["Class"])
    rider_wins["Class"]=np.where((rider_wins["Class"].isna()) & (rider_wins["Race"].str.contains("Prudential Tour")), "1.HC", rider_wins["Class"])
    rider_wins["Class"]=np.where((rider_wins["Class"].isna()) & (rider_wins["Race"].str.contains("Tour de l'Oise")), "2.1", rider_wins["Class"])
    rider_wins["Class"]=np.where((rider_wins["Class"].isna()) & (rider_wins["Race"].str.contains("Trofeo dello Scalatore")), "1.1", rider_wins["Class"])
    rider_wins["Class"]=np.where((rider_wins["Class"].isna()) & (rider_wins["Race"].str.contains("Subida a Urkiola")), "1.1", rider_wins["Class"])
    rider_wins["Class"]=np.where((rider_wins["Class"].isna()) & (rider_wins["Race"].str.contains("Grand Prix de France")), "1.1", rider_wins["Class"])
    rider_wins["Class"]=np.where((rider_wins["Class"].isna()) & (rider_wins["Race"].str.contains("E.O.S. Tallinn")), "1.1", rider_wins["Class"])
    rider_wins["Class"]=np.where((rider_wins["Class"].isna()) & (rider_wins["Race"].str.contains("Halle-ingooigem")), "1.1", rider_wins["Class"])
    rider_wins["Class"]=np.where((rider_wins["Class"].isna()) & (rider_wins["Race"].str.contains("Tour DuPont")), "2.HC", rider_wins["Class"])
    rider_wins["Class"]=np.where((rider_wins["Class"].isna()) & (rider_wins["Race"].str.contains("West Virginia Classic")), "2.1", rider_wins["Class"])
    rider_wins["Class"]=np.where((rider_wins["Class"].isna()) & (rider_wins["Race"].str.contains("Tallinn-Tartu Grand Prix")), "1.1", rider_wins["Class"])
    rider_wins["Class"]=np.where((rider_wins["Class"].isna()) & (rider_wins["Race"].str.contains("Tour of Ireland")), "2.1", rider_wins["Class"])
    rider_wins["Class"]=np.where((rider_wins["Class"].isna()) & (rider_wins["Race"].str.contains("Rominger Classic")), "1.1", rider_wins["Class"])
    rider_wins["Class"]=np.where((rider_wins["Class"].isna()) & (rider_wins["Race"].str.contains("Tour of Iran (Azarbaijan)")), "2.1", rider_wins["Class"])
    rider_wins["Class"]=np.where((rider_wins["Class"].isna()) & (rider_wins["Race"].str.contains("Tour of Iran")), "2.1", rider_wins["Class"])
    rider_wins["Class"]=np.where((rider_wins["Class"].isna()) & (rider_wins["Race"].str.contains("Herald Sun Tour")), "2.1", rider_wins["Class"])
    rider_wins["Class"]=np.where((rider_wins["Class"].isna()) & (rider_wins["Race"].str.contains("Szlakiem Grodów Piastowskich")), "2.1", rider_wins["Class"])
    rider_wins["Class"]=np.where((rider_wins["Class"].isna()) & (rider_wins["Race"].str.contains("Dwars door het Hageland")), "1.1", rider_wins["Class"])
    rider_wins["Class"]=np.where((rider_wins["Class"].isna()) & (rider_wins["Race"].str.contains("Circuito de Getxo-Memorial Hermanos Otxoa")), "1.1", rider_wins["Class"])
    rider_wins["Class"]=np.where((rider_wins["Class"].isna()) & (rider_wins["Race"].str.contains("Tacx Pro Classic")), "1.1", rider_wins["Class"])
    rider_wins["Class"]=np.where((rider_wins["Class"].isna()) & (rider_wins["Race"].str.contains("Trofeo Alcúdia-Can Picafort-Playa de Muro")), "1.1", rider_wins["Class"])
    rider_wins["Class"]=np.where((rider_wins["Class"].isna()) & (rider_wins["Race"].str.contains("Trofeo Migjorn")), "1.1", rider_wins["Class"])
    rider_wins["Class"]=np.where((rider_wins["Class"].isna()) & (rider_wins["Race"].str.contains("Czech Tour")), "2.1", rider_wins["Class"])
    rider_wins["Class"]=np.where((rider_wins["Class"].isna()) & (rider_wins["Race"].str.contains("Vuelta Aragon")), "2.1", rider_wins["Class"])
    rider_wins["Class"]=np.where((rider_wins["Class"].isna()) & (rider_wins["Race"].str.contains("Sharjah Tour")), "2.1", rider_wins["Class"])
    rider_wins["Class"]=np.where((rider_wins["Class"].isna()) & (rider_wins["Race"].str.contains("Vuelta a la Comunidad de Madrid")), "2.1", rider_wins["Class"])
    rider_wins["Class"]=np.where((rider_wins["Class"].isna()) & (rider_wins["Race"].str.contains("de Madrid")), "2.1", rider_wins["Class"])
    rider_wins["Class"]=np.where((rider_wins["Class"].isna()) & (rider_wins["Race"].str.contains("Antwerp Port Epic / Sels Trophy")), "1.1", rider_wins["Class"])
    rider_wins["Class"]=np.where((rider_wins["Class"].isna()) & (rider_wins["Race"].str.contains("Tour du Limousin - Nouvelle Aquitaine")), "2.1", rider_wins["Class"])
    rider_wins["Class"]=np.where((rider_wins["Class"].isna()) & (rider_wins["Race"].str.contains("La Tropicale Amissa Bongo Ondimba")), "2.1", rider_wins["Class"])
    rider_wins["Class"]=np.where((rider_wins["Class"].isna()) & (rider_wins["Race"].str.contains("GP Citta di Rio Saliceto e Correggio")), "1.1", rider_wins["Class"])


    rider_wins["Class"]=np.where((rider_wins["Class"].isna()) & (rider_wins["Race"].str.contains("Tirreno-Adriatico")), "2.UWT", rider_wins["Class"])
    rider_wins["Class"]=np.where((rider_wins["Class"].isna()) & (rider_wins["Race"].str.contains("Tour de Pologne")), "2.UWT", rider_wins["Class"])
    rider_wins["Class"]=np.where((rider_wins["Class"].isna()) & (rider_wins["Race"].str.contains("Tour de Suisse")), "2.UWT", rider_wins["Class"])
    rider_wins["Class"]=np.where((rider_wins["Class"].isna()) & (rider_wins["Race"].str.contains("Tour Down Under")), "2.UWT", rider_wins["Class"])
    rider_wins["Class"]=np.where((rider_wins["Class"].isna()) & (rider_wins["Race"].str.contains("Tour of Beijing")), "2.UWT", rider_wins["Class"])
    rider_wins["Class"]=np.where((rider_wins["Class"].isna()) & (rider_wins["Race"].str.contains("Critérium du Dauphiné")), "2.UWT", rider_wins["Class"])
    rider_wins["Class"]=np.where((rider_wins["Class"].isna()) & (rider_wins["Race"].str.contains("Tour de Romandie")), "2.UWT", rider_wins["Class"])
    rider_wins["Class"]=np.where((rider_wins["Class"].isna()) & (rider_wins["Race"].str.contains("Volta Ciclista a Catalunya")), "2.UWT", rider_wins["Class"])
    rider_wins["Class"]=np.where((rider_wins["Class"].isna()) & (rider_wins["Race"].str.contains("Clasica Ciclista San Sebastian")), "2.UWT", rider_wins["Class"])
    rider_wins["Class"]=np.where((rider_wins["Class"].isna()) & (rider_wins["Race"].str.contains("Critérium du Dauphiné Libéré")), "2.HC", rider_wins["Class"])
    rider_wins["Class"]=np.where((rider_wins["Class"].isna()) & (rider_wins["Race"].str.contains("Deutschland Tour")), "2.HC", rider_wins["Class"])
    rider_wins["Class"]=np.where((rider_wins["Class"].isna()) & (rider_wins["Race"].str.contains("Tour de Pologne")), "2.HC", rider_wins["Class"])
    rider_wins["Class"]=np.where((rider_wins["Class"].isna()) & (rider_wins["Race"].str.contains("Clasica Ciclista San Sebastian")), "2.HC", rider_wins["Class"])
    rider_wins["Class"]=np.where((rider_wins["Class"].isna()) & (rider_wins["Race"].str.contains("Tour de Suisse")), "2.HC", rider_wins["Class"])
    rider_wins["Class"]=np.where((rider_wins["Class"]=="WT") & (rider_wins["Race"].str.contains("Tour de Suisse")), "2.HC", rider_wins["Class"])
    rider_wins["Class"]=np.where((rider_wins["Class"].isna()) & (rider_wins["Race"].str.contains("Tour of Germany")), "2.HC", rider_wins["Class"])
    rider_wins["Class"]=np.where((rider_wins["Class"].isna()) & (rider_wins["Race"].str.contains("Volta Ciclista a Catalunya")), "2.HC", rider_wins["Class"])
    rider_wins["Class"]=np.where((rider_wins["Class"].isna()) & (rider_wins["Race"].str.contains("Tour de Romandie")), "2.HC", rider_wins["Class"])
    rider_wins["Class"]=np.where((rider_wins["Class"].isna()) & (rider_wins["Race"].str.contains("Critérium du Dauphiné")), "2.HC", rider_wins["Class"])
    rider_wins["Class"]=np.where((rider_wins["Class"].isna()) & (rider_wins["Race"].str.contains("Tour Down Under")), "2.HC", rider_wins["Class"])
    rider_wins["Class"]=np.where((rider_wins["Class"].isna()) & (rider_wins["Race"].str.contains("USA Pro Challenge")), "2.HC", rider_wins["Class"])

    rider_wins["Class"]=np.where((rider_wins["Class"].isna()) & (rider_wins["Race"].str.contains("Tour de Taiwan")), "2.1", rider_wins["Class"])
    rider_wins["Class"]=np.where((rider_wins["Class"].isna()) & (rider_wins["Race"].str.contains("Volta a Portugal em Bicicleta")), "2.1", rider_wins["Class"])
    rider_wins["Class"]=np.where((rider_wins["Class"].isna()) & (rider_wins["Race"].str.contains("4 Jours de Dunkerque")), "2.HC", rider_wins["Class"])
    rider_wins["Class"]=np.where((rider_wins["Class"].isna()) & (rider_wins["Race"].str.contains("Midi-Libre")), "2.HC", rider_wins["Class"])
    rider_wins["Class"]=np.where((rider_wins["Class"].isna()) & (rider_wins["Race"].str.contains("Langkawi")), "2.HC", rider_wins["Class"])

    rider_wins["Class"]=np.where((rider_wins["Class"].isna()), "2.2", rider_wins["Class"])
    
    #rider_wins["Class"]=np.where((rider_wins["Class"].isna()) & (rider_wins["Race"].str.contains("Stage")), "2.", rider_wins["Class"])
    #rider_wins["Class"]=np.where((rider_wins["Class"].isna()) & (rider_wins["Race"].str.contains("Prologue")), "2.", rider_wins["Class"])
        
    rider_wins["Class"]=np.where((rider_wins["Class"]=="Olympics"), "1.1", rider_wins["Class"])    
    rider_wins["Class"]=np.where(rider_wins["Class"]=="WC", "1.1", rider_wins["Class"])
    rider_wins["Class"]=np.where(rider_wins["Class"]=="CDM", "1.2", rider_wins["Class"])
    rider_wins["Class"]=np.where(rider_wins["Class"]=="NC", "1.4", rider_wins["Class"])
    rider_wins["Class"]=np.where(rider_wins["Class"]=="JR", "1.4", rider_wins["Class"])
    rider_wins["Class"]=np.where(rider_wins["Class"]=="CC", "1.3", rider_wins["Class"])
    
    return rider_wins

In [8]:
def classify_types(rider_wins):
    # Classify type of race as a single Stage or a several-day Tour (GC = General Classification)
    rider_wins["Type"]=0

    rider_wins["Type"]=np.where(rider_wins["Class"].str[:2]=="1.", "Stage", rider_wins["Type"])
    rider_wins["Type"]=np.where(rider_wins["Class"].str[:2]=="2.", "GC", rider_wins["Type"])

    rider_wins["Type"]=np.where(rider_wins["Class"]=="NC", "Stage", rider_wins["Type"])
    rider_wins["Type"]=np.where(rider_wins["Class"]=="CC", "Stage", rider_wins["Type"])
    rider_wins["Type"]=np.where(rider_wins["Class"]=="JR", "Stage", rider_wins["Type"])

    rider_wins["Type"]=np.where(rider_wins["Class"]=="WC", "Stage", rider_wins["Type"])
    rider_wins["Type"]=np.where(rider_wins["Class"]=="CDM", "Stage", rider_wins["Type"])
    rider_wins["Type"]=np.where(rider_wins["Class"]=="Olympics", "Stage", rider_wins["Type"])

    rider_wins["Type"]=np.where(rider_wins["Race"].str.contains("Stage"), "Stage", rider_wins["Type"])
    rider_wins["Type"]=np.where(rider_wins["Race"].str.contains("Prologue"), "ITT", rider_wins["Type"])
    rider_wins["Type"]=np.where(rider_wins["Race"].str.contains("ITT"), "ITT", rider_wins["Type"])
    
    return rider_wins

In [9]:
def group_tiers(rider_wins):
    # Tier system from 1-4 based on number of UCI points given to riders (1-4 is arbitrary)
    rider_wins["Tier"]=0
    rider_wins["Tier"]=np.where(rider_wins["Class"].str.contains(".UWT"), 1, rider_wins["Tier"])
    rider_wins["Tier"]=np.where(rider_wins["Class"].str.contains(".HC"), 2, rider_wins["Tier"])
    rider_wins["Tier"]=np.where(rider_wins["Class"].str.contains(".1"), 3, rider_wins["Tier"])
    rider_wins["Tier"]=np.where(rider_wins["Class"].str.contains(".2"), 4, rider_wins["Tier"])

    rider_wins["Tier"]=np.where(rider_wins["Class"]=="Olympics", 1, rider_wins["Tier"])
    rider_wins["Tier"]=np.where(rider_wins["Class"]=="WC", 1, rider_wins["Tier"])
    rider_wins["Tier"]=np.where(rider_wins["Class"]=="CDM", 2, rider_wins["Tier"])
    rider_wins["Tier"]=np.where(rider_wins["Class"]=="NC", 4, rider_wins["Tier"])
    rider_wins["Tier"]=np.where(rider_wins["Class"]=="JR", 4, rider_wins["Tier"])
    # Continental Road Race = 3
    rider_wins["Tier"]=np.where((rider_wins["Class"]=="CC") & (rider_wins["Race"].str.contains("Road Race")), 3, rider_wins["Tier"])
    # Continental TT = 4
    rider_wins["Tier"]=np.where((rider_wins["Class"]=="CC") & (rider_wins["Race"].str.contains("ITT")), 4, rider_wins["Tier"])
    # Fixing weird Oceania race name
    rider_wins["Tier"]=np.where((rider_wins["Class"]=="CC") & (rider_wins["Race"].str.contains("en ligne")), 3, rider_wins["Tier"])

    rider_wins["Tier"]=np.where(rider_wins["Class"].str.contains(".3"), 3, rider_wins["Tier"])
    rider_wins["Tier"]=np.where(rider_wins["Class"].str.contains(".4"), 4, rider_wins["Tier"])
    rider_wins["Tier"]=np.where(rider_wins["Class"].str.contains(".5"), 4, rider_wins["Tier"])
    
    return rider_wins

## Webscraping Functions

In [10]:
def WebScrapeTDFData(y_start, y_end):
    print("Processing Tour de France Startlists and Results per year")
    # webscrape data from procyclingstats.com
    years = list(range(y_start, y_end+1))
    startlist = pd.DataFrame(columns=["Rider","Age","Team","Year","BIB"])
    results = pd.DataFrame(columns=["GC","GC-Time","Rider","Team","Year"])
    df_rider_urls = pd.DataFrame(columns=["rider_url"])
    
    url = "https://www.procyclingstats.com/race/tour-de-france/"
    
    # First stage of some years is "stage 1" and others is "prologue"
    prologue = set([2012, 2010, 2007, 2006, 2004, 2003, 2002, 2001])
    
    # Connect to PCS and read table from years in list
    
    # Get Startlist
    for year in years:
        if year in prologue:
            s1="prologue"
        else:
            s1="stage-1"
        print(url+str(year)+"/"+s1+"/result/result")
        page = requests.get(url+str(year)+"/"+s1+"/result/result")
        source = page.text
        soup = BeautifulSoup(source, 'lxml')
        table = soup.find('table')
        html_tables = pd.read_html(table.prettify())
        df = html_tables[0]
        df["Year"] = year
        df = df[["Rider","Age","Team","Year","BIB"]]
        
        startlist = startlist.append(df, ignore_index = True)
        
        # Get Results
        print(url+str(year)+"/gc")
        source = requests.get(url+str(year)+"/gc").text
        soup = BeautifulSoup(source, 'lxml')
        table = soup.find('table')
        html_tables = pd.read_html(table.prettify())
        df = html_tables[0]
        df["Year"] = year
        df = df[["GC","Timelag","Rider","Team","Year"]]
        df.rename(columns={"Timelag":"GC-Time"}, inplace=True)
        results = results.append(df, ignore_index = True)
        
        # Get Rider URL Links    
        bSoup = BeautifulSoup(page.content, 'html.parser')
        links_list = bSoup.find_all('a')

        rider_links = []
        for link in links_list:
            #if 'href="rider/' in link.attrs:
            if 'href' in link.attrs:
                rider_links.append(link)
        df = pd.DataFrame(rider_links, columns=["rider_url"])
        df_rider_urls = df_rider_urls.append(df, ignore_index = True)
        #print(df_rider_urls["rider_url"].unique())
        
    #############################################################################
    
    # Clean Startlist
                    
    # Fix bug where team names appear alongside rider name
    startlist["Rider"] = startlist.apply(lambda x: x["Rider"].replace(x["Team"], "").strip(), axis=1)

    # Correctly format integer values
    startlist["Age"] = startlist["Age"].astype(int)

    # Give value of 1 or 0 whether rider's bib numbers ends in a 1 or not (BIB ending in 1 implies they are team leader)
    startlist['Team Leader'] = startlist['BIB'].astype(str).str[-1]
    startlist['Team Leader'] = startlist['Team Leader'].astype(int)
    startlist['Team Leader'] = np.where(startlist['Team Leader']==1, 1, 0)

    # Remove unecessary column
    del startlist["BIB"]
    
    # Rider Name = Upper
    startlist["Rider"] = startlist["Rider"].str.upper()
    startlist["Rider"] = startlist["Rider"].replace('\s+', ' ', regex=True).str.strip()
    
    startlist["Year"] = startlist["Year"].astype(int)
    
    #############################################################################
    
    # Clean Results
    
    # Fix bug where team names appear alongside rider name
    results["Rider"] = results.apply(lambda x: x["Rider"].replace(x["Team"], "").strip(), axis=1)
    # Removes riders who abandoned on final stage
    results.dropna(subset=['GC'], inplace=True)

    # Correctly format integer values
    results["GC"] = results["GC"].astype(int)

    # Remove "+" sign from GC-Time column
    results["GC-Time"] = results["GC-Time"].str.replace("+","")

    # Calculate number of seconds each rider is behind overall winner
    # Format seperately for riders under and over an hour behind (pandas struggles to cope with differing datetime formats in same column)
    results["Seconds Behind Winner"] = 0
    results["Seconds Behind Winner"] = np.where(results["GC-Time"].str.contains(r'^\d{1}:\d{2}$')==True, pd.to_datetime(results["GC-Time"].astype(str), format='%M:%S', errors='coerce').dt.time, results["Seconds Behind Winner"])
    results["Seconds Behind Winner"] = np.where(results["GC-Time"].str.contains(r'^\d{2}:\d{2}$')==True, pd.to_datetime(results["GC-Time"].astype(str), format='%M:%S', errors='coerce').dt.time, results["Seconds Behind Winner"])
    results["Seconds Behind Winner"] = np.where(results["GC-Time"].str.contains(r'^\d{1}:\d{2}:\d{2}$')==True, pd.to_datetime(results["GC-Time"].astype(str), format='%H:%M:%S', errors='coerce').dt.time, results["Seconds Behind Winner"])

    # Datetime issue with 0, so add 1 second (to be reverted back to 0 later in the process)
    results["Seconds Behind Winner"] = results["Seconds Behind Winner"].astype(str).str.replace("00:00:00", "00:00:01")

    # Add new columns for seconds/minutes/hours
    results["Second Behind Winner"] = pd.to_datetime(results["Seconds Behind Winner"], errors='coerce').dt.second
    results["Minute Behind Winner"] = pd.to_datetime(results["Seconds Behind Winner"], errors='coerce').dt.minute
    results["Hour Behind Winner"] = pd.to_datetime(results["Seconds Behind Winner"], errors='coerce').dt.hour

    # Turn 1 second back to 0 for GC winners
    results["Second Behind Winner"] = np.where((results["Second Behind Winner"]==1) & (results["Minute Behind Winner"]==0) & (results["Hour Behind Winner"]==0), results["Second Behind Winner"]==0, results["Second Behind Winner"])

    # Calculate number of seconds rider was behind that year's winner 
    results["Seconds Behind Winner"] = results["Second Behind Winner"] + 60*results["Minute Behind Winner"] + 3600*results["Hour Behind Winner"]

    # Remove unecessary columns
    del results["Second Behind Winner"]
    del results["Minute Behind Winner"]
    del results["Hour Behind Winner"]
    del results["GC-Time"]
    del results["Team"]

    # Add new GC ranking to fix bug where some riders were given "1001" or "1010" GC pos
    results = results.sort_values(["Year","Seconds Behind Winner"])
    results["GC"] = results.groupby(["Year"])["Seconds Behind Winner"].rank("dense", ascending=True).astype(int)

    # Rider Name = Upper
    results["Rider"] = results["Rider"].str.upper()
    results["Rider"] = results["Rider"].replace('\s+', ' ', regex=True)

    results["Year"] = results["Year"].astype(int)
    
    #############################################################################
    
    # Clean rider_urls
    df_rider_urls.dropna(inplace=True)
    df_rider_urls["rider_url"] = df_rider_urls["rider_url"].astype(str)
    df_rider_urls = df_rider_urls[df_rider_urls["rider_url"].str.contains('rider/')]
    df_rider_urls["riders_name"] = df_rider_urls["rider_url"].str.split('>').str[1].str.replace('</a',"").str.upper().str.strip()
    df_rider_urls["URL"] = df_rider_urls["rider_url"].str.split('>').str[0].str.replace('<a href="rider/',"").str.replace('"',"")
    df_rider_urls.drop_duplicates(subset=['riders_name'], keep='last', inplace=True)
    #print(df_rider_urls["riders_name"].unique())
    #print(startlist["Rider"].unique())
    del df_rider_urls["rider_url"]
    
    # Merge to get rider urls
    tdf_results = startlist.merge(results, on=["Rider","Year"], how='left')
    tdf_train = tdf_results.merge(df_rider_urls, left_on=["Rider"], right_on=["riders_name"], how='left')
    
    del tdf_train["riders_name"]
    
    
    ##Rider Xavier Florencio was on the startlist for the 2010 Tour but didn't start the first stage, 
    ##I have decided to include him as he was meant to start and Cervelo did not field another rider in his place.
    if 2010 in years:
        tdf_train = tdf_train.append({"Rider":"FLORENCIO XAVIER","Age": 30,"Team": "Cervelo Test Team","Year": 2010,"Team Leader": 0,"GC": "DNF","Seconds Behind Winner": np.nan,"URL": "xavier-florencio"}, ignore_index=True)
    
    # Convert Age and Team Leader to Integer values (other fields cannot be converted at this stage due to null values)
    tdf_train["Age"] = tdf_train["Age"].astype(int)
    tdf_train["Team Leader"] = tdf_train["Team Leader"].astype(int)
    
    """On Procyclingstats.com some riders are given high values (>1000) for GC results where they have been DSQ'd for doping
    I have therefore removed riders above with a result over 200 as this is not possible"""
    
    tdf_train["GC"].fillna(0,inplace=True)
    tdf_train["GC"].replace("DNF", 0, inplace=True)
    tdf_train = tdf_train[tdf_train["GC"]<200]
    tdf_train["GC"].replace(0, "DNF",inplace=True)
    
    print("Finished Processing Startlists and Results \n")
    return tdf_train


In [11]:
def WebscrapeRiderData(y_start, y_end):    
    print("Processing Rider Data:")
    rider_results = pd.DataFrame(columns=["URL","Year","Days since last race","AverageResult_2UWT","AverageResult_2HC","BestResult_2UWT","BestResult_2HC","Total_KMs","UCIPoints","StageRaces","OneDayRaces","StageRaceDNFs","OneDayDNFs"])
    rider_info = pd.DataFrame(columns=["URL","Height","Weight","Nationality"])
        
    url = "https://www.procyclingstats.com/rider/"
    
    years = list(range(y_start, y_end+1))
    
    n = tdf_train["URL"].nunique()
    i = 1
    
    for rider_url in tdf_train["URL"].unique():
        print("Processing results for: " + rider_url + " (" + str(i) + "/" + str(n) + ")")
        for year in years:
            html_tables = getTable("https://www.procyclingstats.com/rider.php?xseason="+str(year)+"&sort=date&race=&km1=&zkm1=&pkm1=equal&limit=200&offset=0&topx=&ztopx=&ptopx=smallerorequal&type=&continent=&pnts=&zpnts=&ppnts=equal&level=&rnk=&zrnk=&prnk=equal&exclude_tt=0&racedate=&zracedate=&pracedate=equal&filter=Filter&id="+rider_url+"&p=results")
            df = html_tables[0]
            df["URL"] = rider_url
            df["Year"] = year
            df = df[["URL","Year","Date","Race","Result","Class","KMs","UCI points"]]
            
            df["Result"] = df["Result"].replace("DNF",np.nan)
            df["Result"] = df["Result"].replace("DF",np.nan)
            df["Result"] = df["Result"].replace("DNS",np.nan)
            df["Result"] = df["Result"].replace("OTL",np.nan)
            df["Result"] = df["Result"].replace("DSQ",np.nan)
            df["Result"] = df["Result"].astype(float)
            
            df = classify_races(df)
            df.dropna(subset=["Date"],inplace=True)
            
            df["Date"] = df["Date"].str.replace("-00","-01")
            
            df["StageRaceDNFs"] = np.where((df["Class"].str.contains("2.")) & (df["Result"].isna()), 1, 0)
            df["OneDayDNFs"] = np.where((df["Class"].str.contains("1.")) & (df["Result"].isna()), 1, 0)
                        
            if year==2007:
                df = df[df["Date"]<"2007-07-07"]
                df["Days since last race"] = (pd.to_datetime("2007-07-07") - pd.to_datetime(df["Date"].max())).days
            if year==2008:
                df = df[df["Date"]<"2008-07-05"]
                df["Days since last race"] = (pd.to_datetime("2008-07-05") - pd.to_datetime(df["Date"].max())).days
            if year==2009:
                df = df[df["Date"]<"2009-07-04"]
                df["Days since last race"] = (pd.to_datetime("2009-07-04") - pd.to_datetime(df["Date"].max())).days
            if year==2010:
                df = df[df["Date"]<"2010-07-03"]
                df["Days since last race"] = (pd.to_datetime("2010-07-03") - pd.to_datetime(df["Date"].max())).days
            if year==2011:
                df = df[df["Date"]<"2011-07-02"]
                df["Days since last race"] = (pd.to_datetime("2011-07-02") - pd.to_datetime(df["Date"].max())).days
            if year==2012:
                df = df[df["Date"]<"2012-06-30"]
                df["Days since last race"] = (pd.to_datetime("2012-06-30") - pd.to_datetime(df["Date"].max())).days
            if year==2013:
                df = df[df["Date"]<"2013-06-29"]
                df["Days since last race"] = (pd.to_datetime("2013-06-29") - pd.to_datetime(df["Date"].max())).days
            if year==2014:
                df = df[df["Date"]<"2014-07-05"]
                df["Days since last race"] = (pd.to_datetime("2014-07-05") - pd.to_datetime(df["Date"].max())).days
            if year==2015:
                df = df[df["Date"]<"2015-07-04"]
                df["Days since last race"] = (pd.to_datetime("2015-07-04") - pd.to_datetime(df["Date"].max())).days
            if year==2016:
                df = df[df["Date"]<"2016-07-02"]
                df["Days since last race"] = (pd.to_datetime("2016-07-02") - pd.to_datetime(df["Date"].max())).days
            if year==2017:
                df = df[df["Date"]<"2017-07-01"]
                df["Days since last race"] = (pd.to_datetime("2017-07-01") - pd.to_datetime(df["Date"].max())).days
            if year==2018:
                df = df[df["Date"]<"2018-07-07"]
                df["Days since last race"] = (pd.to_datetime("2018-07-07") - pd.to_datetime(df["Date"].max())).days
            if year==2019:
                df = df[df["Date"]<"2019-07-06"]
                df["Days since last race"] = (pd.to_datetime("2019-07-06") - pd.to_datetime(df["Date"].max())).days
            if year==2020:
                df = df[df["Date"]<"2020-08-29"]
                df["Days since last race"] = (pd.to_datetime("2020-08-29") - pd.to_datetime(df["Date"].max())).days
            if year==2021:
                df = df[df["Date"]<"2021-06-26"]
                df["Days since last race"] = (pd.to_datetime("2021-06-26") - pd.to_datetime(df["Date"].max())).days
            
            # Store DNF values to be merged in later (removal of individual stage results requires this)
            df_dnfs = df.groupby(["URL","Year"]).agg(StageRaceDNFs=('StageRaceDNFs', 'sum'),
                                                     OneDayDNFs=('OneDayDNFs', 'sum')).reset_index()
            del df["StageRaceDNFs"]
            del df["OneDayDNFs"]
            
            # Display races which do not fit into this class category
            if not df[df["Class"]=="2.UWT|1.UWT|2.HC|1.HC|2.1|1.1|2.2|1.2"].empty:
                display(df[df["Class"]=="2.UWT|1.UWT|2.HC|1.HC|2.1|1.1|2.2|1.2"])
                        
            df["UCI points"] = df["UCI points"].replace("-",np.nan).astype(float)
            df["Total KMs"] = df["KMs"].sum()
            df["Total UCI Points"] = df["UCI points"].sum()
            
            # Remove stages results of stage races, and other classifications
            df = df[(~df["Race"].str.contains("Stage|Prologue|ITT|TTT|Mountains classification|Points classification|Youth classification"))]
            df["Class"] = np.where((df["KMs"].isna()) & (df["Class"].str.contains("UWT")),"2.UWT",df["Class"])
            df["Class"] = np.where((~df["KMs"].isna()) & (df["Class"].str.contains("UWT")),"1.UWT",df["Class"])
            df["Class"] = np.where((df["KMs"].isna()) & (df["Class"].str.contains("HC")),"2.HC",df["Class"])
            df["Class"] = np.where((~df["KMs"].isna()) & (df["Class"].str.contains("HC")),"1.HC",df["Class"])
            
            # Create cols to be used for aggregation of results per tier (UWT, HC, and Other)
            df["2.UWT_Result"] = np.where(df["Class"]=="2.UWT", 1, np.nan)*df["Result"]
            df["2.HC_Result"] = np.where(df["Class"]=="2.HC", 1, np.nan)*df["Result"]
            df["2.Other_Result"] = np.where(df["Class"]!="2.UWT|2.HC", 1, np.nan)*df["Result"]
            
            """ Remove results over 300 as they are not possible in any race.
            Riders with results over 300 are likely due to them doping as Procyclingstats.com deals with this by 
            giving them an unreasonably high value for their result.
            """
            df = df[df["Result"]<300]
            

            df = df.groupby(['URL','Year','Days since last race']).agg(AverageResult_2UWT=('2.UWT_Result', 'mean'),
                                                AverageResult_2HC=('2.HC_Result', 'mean'),
                                                AverageResult_2Other=('2.Other_Result', 'mean'),
                                                BestResult_2UWT=('2.UWT_Result', 'min'),
                                                BestResult_2HC=('2.HC_Result', 'min'),
                                                BestResult_2Other=('2.Other_Result', 'min'),
                                                Total_KMs=('Total KMs','max'),
                                                UCIPoints=('Total UCI Points','max'),
                                                StageRaces=('Class', lambda x: x.str.contains("2.").sum()),
                                                OneDayRaces=('Class', lambda x: x.str.contains("1.").sum())).reset_index()
            # Merge in number of DNFs
            df = df.merge(df_dnfs, on=["URL","Year"], how='left')
                                                                                              
            rider_results = rider_results.append(df, ignore_index = True)   
            
        i=i+1

    print("Finished Processing Rider Data \n")
    return rider_results


In [12]:
def WebscrapeRiderInfo():
    print("Processing Rider Info:")
    url = "https://www.procyclingstats.com/rider/"
    rider_info = pd.DataFrame(columns=["URL","Height","Weight","Nationality"])
    
    n = tdf_train["URL"].nunique()
    i=1
    for rider_url in tdf_train["URL"].unique():
        print("Processing info for: " + rider_url + " (" + str(i) + "/" + str(n) + ")")
        height = getHeight(url+rider_url)
        weight = getWeight(url+rider_url)
        nat = getNat(url+rider_url)
        df = pd.DataFrame ({'URL': [rider_url], 'Height': [height], 'Weight': [weight], 'Nationality': [nat]}, columns=['URL','Height','Weight','Nationality'])
        rider_info = rider_info.append(df, ignore_index = True)
        
        i=i+1

    print("Finished Processing Rider Info \n")
    return rider_info


In [13]:
def WebscrapeRiderGTs(y_start, y_end):
    print("Processing Rider Grand Tour Results:")
    
    years = list(range(y_start, y_end+1))
    
    url = "https://www.procyclingstats.com/rider/"
    
    rider_gts = pd.DataFrame(columns=["URL","Year","numTours","numGiros","numVueltas","BestGTFinish","BestTourFinish",
                                     "BestGTStageFinish","FirstGT","LastGT","FirstTour","LastTour","NumGTDNFs"])
    
    n = tdf_train["URL"].nunique()
    i=1
    for rider_url in tdf_train["URL"].unique():
        print("Processing Grand-Tour results for: " + rider_url + " (" + str(i) + "/" + str(n) + ")")
        
        for year in years:
            html_tables = getTable(url+rider_url+"/statistics/grand-tour-starts")
            df = html_tables[0]
            df["URL"] = rider_url
            df["Year"] = year
            df = df[["URL","Year","Season","Grand tour","GC","Best stage result"]]
                        
            if year==2020:
                #Giro took place after Tour in 2020
                df["Giro"] = np.where((df["Grand tour"]=="Giro d'Italia") & (df["Season"]<year), 1, 0)
                df["Tour"] = np.where((df["Grand tour"]=="Tour de France") & (df["Season"]<year), 1, 0)
                df["Vuelta"] = np.where(((df["Grand tour"]=="Vuelta a España") | (df["Grand tour"]=="La Vuelta ciclista a España")) & (df["Season"]<year), 1, 0)
                
            else:
                #Giro takes place before Tour, Vuelta takes place after, so +1 for year in Giro.
                df["Giro"] = np.where((df["Grand tour"]=="Giro d'Italia") & (df["Season"]<year+1), 1, 0)
                df["Tour"] = np.where((df["Grand tour"]=="Tour de France") & (df["Season"]<year), 1, 0)
                df["Vuelta"] = np.where(((df["Grand tour"]=="Vuelta a España") | (df["Grand tour"]=="La Vuelta ciclista a España")) & (df["Season"]<year), 1, 0)
            
            df["GC"] = df["GC"].replace("DNF",np.nan)
            df["GC"] = df["GC"].replace("DNS",np.nan)
            df["GC"] = df["GC"].replace("OTL",np.nan)
            df["GC"] = df["GC"].replace("DSQ",np.nan)
            df = df[df["GC"]!="In progress"]
            df["GC"] = df["GC"].astype(float)
            
            df["TourRes"] = df["Tour"]*df["GC"]
            df["TourRes"] = df["TourRes"].replace(0,np.nan)
            df["TourYr"] = df["Tour"]*df["Season"]
            df["TourYr"] = df["TourYr"].replace(0,np.nan)
            
            if year==2020:
                df = df[df["Season"]<year]
            else:
                df = df[(df["Season"]<year) | ((df["Grand tour"]=="Giro d'Italia") & (df["Season"]<year+1))]
      
            df = df.groupby(['URL','Year']).agg(numTours=('Tour', 'sum'),
                                                numGiros=('Giro', 'sum'),
                                                numVueltas=('Vuelta', 'sum'),
                                                BestGTFinish=('GC', 'min'),
                                                BestTourFinish=('TourRes', 'min'),
                                                BestGTStageFinish=('Best stage result', 'min'),
                                                FirstGT = ('Season', 'min'),
                                                LastGT = ('Season', 'max'),
                                                FirstTour = ('TourYr', 'min'),
                                                LastTour = ('TourYr', 'max'),
                                                NumGTDNFs=('GC', lambda x: x.isna().sum())).reset_index()
                                                
            rider_gts = rider_gts.append(df, ignore_index = True)
            
        i=i+1
    
    rider_gts["numGTs"] = rider_gts["numTours"] + rider_gts["numGiros"] + rider_gts["numVueltas"]

    print("Finished Processing Rider Grand-Tour Results. \n")
    return rider_gts

In [14]:
def RunAll(test_mode, y_start, y_end):
    global tdf_train
    y_start = y_start
    y_end = y_end
    start = time.time()
    tdf_train = WebScrapeTDFData(y_start, y_end)
    if test_mode==True:
        tdf_train = tdf_train[(tdf_train["URL"]=="adam-yates") | (tdf_train["URL"]=="jose-vicente-garcia-acosta") 
                              | (tdf_train["URL"]=="levi-leipheimer") | (tdf_train["URL"]=="mark-cavendish")]
    elif test_mode==False:
        pass
    else:
        print("Invalid Choice for test_mode!")

    rider_results = WebscrapeRiderData(y_start, y_end)
    rider_info = WebscrapeRiderInfo()
    rider_gts = WebscrapeRiderGTs(y_start, y_end)

    # Merge
    rider_data = rider_results.merge(rider_info, on=["URL"], how='left')
    rider_data_gts = rider_data.merge(rider_gts, on=["URL","Year"], how='left')
    tdf_final = tdf_train.merge(rider_data_gts, on=["URL","Year"], how='left')
    
    # Sort by Year and GC Position
    tdf_final = tdf_final.sort_values(["Year","Seconds Behind Winner"])
    
    end = time.time()
    print("Run Time: " + str((end - start)/60) + " minutes.")
    
    return tdf_final

# Run

In [15]:
tdf_final = RunAll(False, 2021, 2021)

Processing Tour de France Startlists and Results per year
https://www.procyclingstats.com/race/tour-de-france/2021/stage-1/result/result
https://www.procyclingstats.com/race/tour-de-france/2021/gc
Finished Processing Startlists and Results 

Processing Rider Data:
Processing results for: julian-alaphilippe (1/184)
Processing results for: michael-matthews (2/184)
Processing results for: primoz-roglic (3/184)
Processing results for: jack-haig (4/184)
Processing results for: wilco-kelderman (5/184)
Processing results for: tadej-pogacar (6/184)
Processing results for: david-gaudu (7/184)
Processing results for: sergio-higuita (8/184)
Processing results for: bauke-mollema (9/184)
Processing results for: geraint-thomas (10/184)
Processing results for: johan-esteban-chaves (11/184)
Processing results for: pello-bilbao (12/184)
Processing results for: jonas-vingegaard-rasmussen (13/184)
Processing results for: aleksey-lutsenko (14/184)
Processing results for: enric-mas (15/184)
Processing resu

Processing results for: pierre-luc-perichon (165/184)
Processing results for: marco-haller (166/184)
Processing results for: ide-schelling (167/184)
Processing results for: bryan-coquard (168/184)
Processing results for: clement-russo (169/184)
Processing results for: daniel-mclay (170/184)
Processing results for: christopher-froome (171/184)
Processing results for: omer-goldstein (172/184)
Processing results for: mads-pedersen (173/184)
Processing results for: reto-hollenstein (174/184)
Processing results for: rick-zabel (175/184)
Processing results for: tony-martin (176/184)
Processing results for: sepp-kuss (177/184)
Processing results for: mike-teunissen (178/184)
Processing results for: amund-grondahl-jansen (179/184)
Processing results for: marc-hirschi (180/184)
Processing results for: marc-soler (181/184)
Processing results for: ignatas-konovalovas (182/184)
Processing results for: jasha-sutterlin (183/184)
Processing results for: cyril-lemoine (184/184)
Finished Processing Rid

Processing info for: benoit-cosnefroy (159/184)
Processing info for: georg-zimmermann (160/184)
Processing info for: dmitriy-gruzdev (161/184)
Processing info for: ion-izagirre (162/184)
Processing info for: andre-greipel (163/184)
Processing info for: edward-theuns (164/184)
Processing info for: pierre-luc-perichon (165/184)
Processing info for: marco-haller (166/184)
Processing info for: ide-schelling (167/184)
Processing info for: bryan-coquard (168/184)
Processing info for: clement-russo (169/184)
Processing info for: daniel-mclay (170/184)
Processing info for: christopher-froome (171/184)
Processing info for: omer-goldstein (172/184)
Processing info for: mads-pedersen (173/184)
Processing info for: reto-hollenstein (174/184)
Processing info for: rick-zabel (175/184)
Processing info for: tony-martin (176/184)
Processing info for: sepp-kuss (177/184)
Processing info for: mike-teunissen (178/184)
Processing info for: amund-grondahl-jansen (179/184)
Processing info for: marc-hirschi (

Processing Grand-Tour results for: jacopo-guarnieri (119/184)
Processing Grand-Tour results for: petr-vakoc (120/184)
Processing Grand-Tour results for: dan-martin (121/184)
Processing Grand-Tour results for: jeremy-cabot (122/184)
Processing Grand-Tour results for: tao-geoghegan-hart (123/184)
Processing Grand-Tour results for: jonathan-castroviejo (124/184)
Processing Grand-Tour results for: jonas-koch (125/184)
Processing Grand-Tour results for: max-walscheid (126/184)
Processing Grand-Tour results for: daniel-oss (127/184)
Processing Grand-Tour results for: roger-kluge (128/184)
Processing Grand-Tour results for: caleb-ewan (129/184)
Processing Grand-Tour results for: warren-barguil (130/184)
Processing Grand-Tour results for: tim-declercq (131/184)
Processing Grand-Tour results for: mark-cavendish (132/184)
Processing Grand-Tour results for: michael-morkov (133/184)
Processing Grand-Tour results for: luke-durbridge (134/184)
Processing Grand-Tour results for: victor-campenaerts (1

# Merge All DFs

# Final Output and Write to csv

In [16]:
tdf_final

,Rider,Age,Team,Year,Team Leader,GC,Seconds Behind Winner,URL,Days since last race,AverageResult_2UWT,...,numVueltas,BestGTFinish,BestTourFinish,BestGTStageFinish,FirstGT,LastGT,FirstTour,LastTour,NumGTDNFs,numGTs
5,POGAČAR TADEJ,22,UAE-Team Emirates,2021,1,1,0.0,tadej-pogacar,6,1.666667,...,1,1.0,1.0,1,2019,2020,2020.0,2020.0,0.0,2
12,VINGEGAARD JONAS,24,Team Jumbo-Visma,2021,0,2,320.0,jonas-vingegaard-rasmussen,20,33.000000,...,1,46.0,NaN,14,2020,2020,NaN,NaN,0.0,1
21,CARAPAZ RICHARD,28,INEOS Grenadiers,2021,0,3,423.0,richard-carapaz,13,13.666667,...,3,1.0,13.0,1,2017,2020,2020.0,2020.0,0.0,6
39,O'CONNOR BEN,25,AG2R Citroën Team,2021,0,4,602.0,ben-o-connor,18,12.250000,...,1,20.0,NaN,1,2018,2020,NaN,NaN,1.0,4
4,KELDERMAN WILCO,30,BORA - hansgrohe,2021,0,5,613.0,wilco-kelderman,20,6.333333,...,4,3.0,32.0,2,2013,2020,2015.0,2019.0,2.0,11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
178,JANSEN AMUND GRØNDAHL,27,Team BikeExchange,2021,0,DNF,NaN,amund-grondahl-jansen,15,NaN,...,0,128.0,128.0,1,2018,2020,2018.0,2020.0,0.0,3
180,SOLER MARC,27,Movistar Team,2021,0,DNF,NaN,marc-soler,13,42.500000,...,3,9.0,21.0,1,2017,2021,2018.0,2020.0,1.0,7
181,KONOVALOVAS IGNATAS,35,Groupama - FDJ,2021,0,DNF,NaN,ignatas-konovalovas,6,102.000000,...,2,90.0,127.0,1,2009,2020,2010.0,2017.0,4.0,10
182,SÜTTERLIN JASHA,28,Team DSM,2021,0,DNF,NaN,jasha-sutterlin,6,94.500000,...,1,55.0,108.0,12,2016,2020,2017.0,2017.0,0.0,4


In [17]:
tdf_final[tdf_final["URL"]=="mark-cavendish"]

,Rider,Age,Team,Year,Team Leader,GC,Seconds Behind Winner,URL,Days since last race,AverageResult_2UWT,...,numVueltas,BestGTFinish,BestTourFinish,BestGTStageFinish,FirstGT,LastGT,FirstTour,LastTour,NumGTDNFs,numGTs
131,CAVENDISH MARK,36,Deceuninck - Quick Step,2021,0,138,16454.0,mark-cavendish,13,NaN,...,2,127.0,130.0,1,2007,2018,2007.0,2018.0,9.0,19


In [18]:
tdf_final.to_csv(dPath+"tdf_train_2021_f.csv", index=False)

# Tour Start dates & End dates
- 2006-07-01 - 2006-07-23
- 2007-07-07 - 2007-07-29
- 2008-07-05 - 2008-07-27
- 2009-07-04 - 2009-07-26
- 2010-07-03 - 2010-07-25
- 2011-07-02 - 2011-07-24
- 2012-06-30 - 2012-07-22
- 2013-06-29 - 2013-07-21
- 2014-07-05 - 2014-07-27
- 2015-07-04 - 2015-07-26
- 2016-07-02 - 2016-07-24
- 2017-07-01 - 2017-07-23
- 2018-07-07 - 2018-07-29
- 2019-07-06 - 2019-07-28
- 2020-08-29 - 2020-09-20


# Extra Info
A ‘1’ at the front means it’s a one day race and a ‘2’ means it’s a multi stage event which can be anything from 2 days to 21 days long.”

Class Order:
- UWT
- HC
- 1
- 2
Other levels:
- WC (World Champs)
- NC (National Champs)
- Olypmics
- CC (Continental Championship - Cyclo Cross)
- CDM (World Cup - Cyclo Cross)
- HC (? Only occured for one year 1998/99)
- HIS (? Only in 2010)
- JR (Regional Games - Cyclo Cross)
- PT (? Only for 2001-2010)
- WT (World Tour - Road/Cyclo Cross)

- JO = Olympic Games (it stands for Jeux Olympiques, the UCI being a mainly Francophone organization)
- CM = World Championships (Championnat Mondial)
- GT = Grand Tour
- CDM = World Cup (Coupe du Monde)
- HC = Hors Categorie (Beyond or above category)
- CN = National Championships
- CC = Continental Championships
- CMM = World Masters Championships
- JR = Regional Games
- SC = Super Calendar (Juniors)
- S = Special (No UCI points)
- NE = National Event (No UCI points)

The Elite Men are given values between *.HC (Hors Categorie, one level below grand tour) and *.6 (a UCI sanctioned race with no UCI points on offer). As explained above, * is either 1 or 2, depending on whether the race is single or multi-day. The higher the numbers, the lower the classification of the race in terms of UCI points, prize money, and number of division I teams allowed to start. Thus, a *.1 race will typically attract a stronger field than a *.3 race.

The Elite Women have just two standard classifications: *.9.1 and *.9.2. Similarly, the U23 men have two classifications; *.7.1 and *.7.2. Junior Men are either *.8 or SC, the latter standing for Super Calendar races which contribute to the annual Junior Men's points score. Junior Women have the *.10 classification, but to our knowledge there is only one UCI junior women's even on the calendar, apart from the World Championships.

In [19]:
tdf_train.groupby(["Year"])["Team"].nunique()

Year
2021    23
Name: Team, dtype: int64

In [20]:
tdf_train.groupby(["Year"])["Rider"].nunique()

Year
2021    184
Name: Rider, dtype: int64

In [21]:
tdf_final.columns

Index(['Rider', 'Age', 'Team', 'Year', 'Team Leader', 'GC',
       'Seconds Behind Winner', 'URL', 'Days since last race',
       'AverageResult_2UWT', 'AverageResult_2HC', 'BestResult_2UWT',
       'BestResult_2HC', 'Total_KMs', 'UCIPoints', 'StageRaces', 'OneDayRaces',
       'StageRaceDNFs', 'OneDayDNFs', 'AverageResult_2Other',
       'BestResult_2Other', 'Height', 'Weight', 'Nationality', 'numTours',
       'numGiros', 'numVueltas', 'BestGTFinish', 'BestTourFinish',
       'BestGTStageFinish', 'FirstGT', 'LastGT', 'FirstTour', 'LastTour',
       'NumGTDNFs', 'numGTs'],
      dtype='object')